In [1]:
import tensorflow as tf
import numpy as np
from datetime import datetime, timedelta
import boto3 
from botocore import UNSIGNED
from botocore.config import Config
import matplotlib.patches as mpatches
import io
import os

from tfrecordFormat import get_dataset_grib,graphGrib,grib_png,parsegrib#,plot_animation_grib

2022-03-09 19:33:25.431727: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-09 19:33:25.431826: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
_MM_PER_HOUR_INCREMENT = 1/32.
_MAX_MM_PER_HOUR = 128.
_INT16_MASK_VALUE = -1

In [3]:
def parsegrib(element):
    print('parsing')
    
    '''
    "radar": bytes_feature(serialize_array(example)), # tf.string
        "sample_prob": float_feature(1.),  # tf.float32
        "lonmin": int64_feature(int(metadata['GRIB_longitudeOfFirstGridPointInDegrees'])),#(metadata[0].latlong_box[0]), # tf.int64
        "lonmax": int64_feature(int(metadata['GRIB_longitudeOfLastGridPointInDegrees'])),#(metadata[0].latlong_box[1]), # tf.int64
        "latmin": int64_feature(int(metadata['GRIB_latitudeOfFirstGridPointInDegrees'])),#(metadata[0].latlong_box[2]), # tf.int64
        "latmax": int64_feature(int(metadata['GRIB_latitudeOfLastGridPointInDegrees']))#(metadata[0].latlong_box[3]), # tf.int64
 
    '''
    features = {
      'latmin': tf.io.FixedLenFeature([], tf.int64), 
      'latmax':tf.io.FixedLenFeature([], tf.int64),
      'radar' : tf.io.FixedLenFeature([], tf.string),
      'latitude' : tf.io.FixedLenFeature([], tf.string),
      'longitude' : tf.io.FixedLenFeature([], tf.string),
        
      'lonmin':tf.io.FixedLenFeature([], tf.int64),
      'lonmax':tf.io.FixedLenFeature([], tf.int64),
      'end_time_timestamp':tf.io.FixedLenFeature([], tf.int64)  
        
    }
    
    
    result = tf.io.parse_single_example(element, features) 
    radar_bytes = result['radar']#result.pop("radar")
    shape = (3500, 7000)

    radar_float32 = tf.reshape(tf.io.parse_tensor(radar_bytes, tf.float32), shape)
    print('radar bytes')
    print(radar_bytes)
    result['latitude'] = tf.io.parse_tensor(result['latitude'], tf.float64) 
    result['longitude'] = tf.io.parse_tensor(result['longitude'], tf.float64)

    result['latitude'] = tf.cast(result['latitude'], tf.float32) 
    result['longitude'] = tf.cast(result['longitude'], tf.float32)


    mask = tf.not_equal(radar_float32, _INT16_MASK_VALUE)

    #radar = radar_float64 * _MM_PER_HOUR_INCREMENT
    radar = tf.clip_by_value(
      radar_float32, _INT16_MASK_VALUE * _MM_PER_HOUR_INCREMENT, _MAX_MM_PER_HOUR)
    
    
    '''
    #get our 'feature'-- our image -- and reshape it appropriately
    feature = tf.io.parse_tensor(radar, out_type=tf.int16)
    feature = tf.reshape(feature, shape=[256,256,1])
    return (feature, label)
    '''
    #result["radar_frames"] = radar_float64
    #result["radar_mask"] = mask
    #result['x'] = tf.io.parse_tensor(result['x'], tf.float64) 
    #result['y'] = tf.io.parse_tensor(result['y'], tf.float64)
    #print(result)
    
    return {'radar_frames':radar,
            "radar_mask":mask,
            'latmin':result['latmin'],
            'latmax':result['latmax'],
            'lonmin':result['lonmin'], 
            'lonmax': result['lonmax'],
            'longitude':result['longitude'],
            'latitude':result['latitude'],
            'end_time_timestamp':result['end_time_timestamp']}


In [4]:
#s3://lumo-app/ground-truths/grib_24hr/
def grib_png(row,map_extent = [-106.0, -78.05576, 30.0, 47.603355],figsize=None,
                   vmin=0, vmax=15, cmap="jet", **imshow_args):
  
    #s3Pre = 
    s3_client = boto3.client( 's3',
          aws_access_key_id="",
          aws_secret_access_key="",
          region_name="us-east-1"
    )
      
    response = s3_client.list_objects_v2(Bucket="lumo-app", Prefix="test-predictions/grib_24hr/")
    if 'Contents' in response:
        for object in response['Contents']:
            print('Deleting', object['Key'])
            s3_client.delete_object(Bucket="lumo-app", Key=object['Key'])
  
    #for i in range(field.shape[0]):
    #print(f'img : {i}')
    globe = cartopy.crs.Globe(semimajor_axis=6371200.0) 
    proj = cartopy.crs.Stereographic(central_latitude=90, 
                    central_longitude= -105,
                    true_scale_latitude=60, 
                    globe=globe)

    fig = plt.figure(figsize=(10, 10))
    crs = cartopy.crs.PlateCarree()
    ax = plt.axes(projection=crs)
      
    ax.set_extent(map_extent, crs=crs)
    
    # draw coastlines, state and country boundaries, edge of map.
    ax.coastlines()
    ax.add_feature(cartopy.feature.BORDERS)
    lakes = cartopy.feature.NaturalEarthFeature('physical', 'lakes', '10m')
    ax.add_feature(lakes, edgecolor='gold', linewidth=0.25)

    waterloo = mpatches.Circle((-80.516670, 43.466667), 0.065, linestyle='solid', edgecolor='black', facecolor='gold')
    ax.add_patch(waterloo)
    big_loo = mpatches.Circle((-80.516670, 43.466667), 0.094, linestyle='solid', edgecolor='gold', facecolor='none')
    ax.add_patch(big_loo)

    data = row['radar_frames']#[500:950, 800:1100] 
    lons = np.array(row['longitude'])
    lats = np.array(row['latitude'])
    
    convert = Proj(proj.proj4_init)
    X, Y = np.meshgrid(lons, lats)
    X, Y = convert(X, Y)
    print(X.shape, Y.shape)
    print(X.min(), X.max(), Y.min(), Y.max())  
    
    cs = ax.imshow(data, extent=[X.min(), X.max(), Y.min(), Y.max()], cmap='jet', vmin=vmin, vmax=vmax, origin='upper', interpolation='none', transform=proj)
    # add colorbar.
    cbar = plt.colorbar(cs, orientation='horizontal')
    cbar.set_label("millimeter")

    plt.show()

    img_data = io.BytesIO()
    fig.savefig(img_data, format='png', bbox_inches='tight')
    img_data.seek(0)

    session = boto3.Session(
        aws_access_key_id="",
        aws_secret_access_key="",
    )
    s3 = session.resource('s3')
    bucket = s3.Bucket("lumo-app")

    datetime_str = datetime.datetime.utcfromtimestamp(row['end_time_timestamp']).strftime("%Y-%m-%d-%H%M")
    print(datetime_str)
    #index_str = f'0{str(i)}' if i < 10 else str(i)
    print('creating img')
    bucket.put_object(Body=img_data, ContentType='image/png', Key= "ground-truths/mrms_grib/" +  datetime_str)
                      #"test-predictions/mrms_grib/" +  datetime_str)
#ground-truths/grib_24hr/

In [5]:
def getS3FileNames(gribGroundTruthsPath):
  s3_client = boto3.client( 's3',
        aws_access_key_id="",
        aws_secret_access_key="",
        region_name="us-east-1"
    )
      
  response = s3_client.list_objects_v2(Bucket="lumo-app", Prefix=gribGroundTruthsPath)

  #result = s3.list_objects_v2(Bucket=bucket, Prefix=prefix)
  filenames = []
  if 'Contents' not in response:
    print('no data')
    return []
  for item in response['Contents']:
    files = item['Key']
    if( ".gz" in files):
      #print(files)
      filenames.append("s3://lumo-app/"+files) 

  return filenames
 

In [6]:
#read tf record and call function
w_specBounds = [-82.2, -79.35576, 42.7, 44.603355]

gribGroundTruthsPath = "ground-truths/grib_1hr/"
gribGroundTruthsPath = "ground-truths/grib_24hr/"
#s3://lumo-app/test-predictions/grib_24hr/
# get files list
filenames =  getS3FileNames("test-predictions/grib_24hr/")
print(filenames)
# first check file is retrived w graph and get data set then plot
if( len(filenames) < 4):
  print('not enough files')

dataset = tf.data.TFRecordDataset(filenames[0], compression_type="GZIP") 
dataset = dataset.map(parsegrib)    
print(dataset)
grib_row = next(iter(dataset)) 
grib_row

['s3://lumo-app/test-predictions/grib_24hr/MRMS_RadarOnly_QPE_24H_00.00_20220307-000000.tfrecord.gz', 's3://lumo-app/test-predictions/grib_24hr/MRMS_RadarOnly_QPE_24H_00.00_20220307-010000.tfrecord.gz', 's3://lumo-app/test-predictions/grib_24hr/MRMS_RadarOnly_QPE_24H_00.00_20220307-020000.tfrecord.gz', 's3://lumo-app/test-predictions/grib_24hr/MRMS_RadarOnly_QPE_24H_00.00_20220307-030000.tfrecord.gz']


2022-03-09 19:33:36.000786: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-03-09 19:33:36.000851: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-09 19:33:36.000874: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (0e29016c822e): /proc/driver/nvidia/version does not exist
2022-03-09 19:33:36.001329: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


parsing
radar bytes
Tensor("ParseSingleExample/ParseExample/ParseExampleV2:7", shape=(), dtype=string)
<MapDataset element_spec={'radar_frames': TensorSpec(shape=(3500, 7000), dtype=tf.float32, name=None), 'radar_mask': TensorSpec(shape=(3500, 7000), dtype=tf.bool, name=None), 'latmin': TensorSpec(shape=(), dtype=tf.int64, name=None), 'latmax': TensorSpec(shape=(), dtype=tf.int64, name=None), 'lonmin': TensorSpec(shape=(), dtype=tf.int64, name=None), 'lonmax': TensorSpec(shape=(), dtype=tf.int64, name=None), 'longitude': TensorSpec(shape=<unknown>, dtype=tf.float32, name=None), 'latitude': TensorSpec(shape=<unknown>, dtype=tf.float32, name=None), 'end_time_timestamp': TensorSpec(shape=(), dtype=tf.int64, name=None)}>


UnimplementedError: File system scheme 's3' not implemented (file: 's3://lumo-app/test-predictions/grib_24hr/MRMS_RadarOnly_QPE_24H_00.00_20220307-000000.tfrecord.gz') [Op:IteratorGetNext]

In [ ]:
#data = get_dataset_grib(filenames[0]) 
#gribfiles/24hr/MRMS_RadarOnly_QPE_24H_00.00_20220307-010000.grib2
#gribfiles/24hr/MRMS_RadarOnly_QPE_24H_00.00_20220307-030000.tfrecord.gz
fn = "./gribfiles/24hrMRMS_RadarOnly_QPE_24H_00.00_20220307-000000.tfrecord.gz" 
   
dataset = tf.data.TFRecordDataset(fn, compression_type="GZIP") 
   
print(dataset) 
dataset = dataset.map(parsegrib) 
print(dataset)
grib_row = next(iter(dataset)) 

In [ ]:
graphGrib(grib_row,w_specBounds)

# 1 then do in parallel w a function w png conversion
#plot_animation_grib()

#grib_png(grib_row,w_specBounds)